# ANN - PyTorch

In [70]:
import pandas as pd
import numpy as np
import pickle
import networkx as nx
import matplotlib.pyplot as plt

import math
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from random import shuffle

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
# Import database
database = pickle.load( open( "Mod_AnyT_DB.p", "rb" ) )

In [3]:
x_diameters = []
y_res_index = []
for i in range(len(database)):
    x_diameters.append(database.loc[i]['Diams'])
    y_res_index.append(database.loc[i]['avgPrPa'])

In [50]:
X_train, X_test, y_train, y_test = train_test_split(x_diameters, y_res_index, random_state=1, test_size=0.2 )
print(len(X_train), len(y_test))

4000 1000


In [51]:
X_train, X_test, y_train, y_test = torch.tensor(X_train, dtype = torch.float32), torch.tensor(X_test, dtype = torch.float32), torch.tensor(y_train, dtype = torch.float32), torch.tensor(y_test, dtype = torch.float32)

### ANN model definition

In [94]:
class MLP(nn.Module):
    def __init__(self, in_features, h_sizes, out_size):
        super(MLP, self).__init__()
        self.hidden_linear = nn.ModuleList()
        self.hidden_dropout = nn.ModuleList()
        self.h_len = len(h_sizes)
        
        #Add initial layer
        self.hidden_linear.append(nn.Linear(in_features, h_sizes[0]))
        self.hidden_dropout.append(nn.Dropout(p=0.5))
        
        #Add hidden layers        
        for k in range(len(h_sizes)-1):
            self.hidden_linear.append(nn.Linear(h_sizes[k], h_sizes[k+1]))
            self.hidden_dropout.append(nn.Dropout(p=0.5))
        
        #Add output layer
        self.hidden_linear.append(nn.Linear(h_sizes[-1], out_size))
        
        
    def forward(self, x):
        y = x
        for i in range(len(self.hidden_dropout)):
            y = self.hidden_linear[i](y)
            y = self.hidden_dropout[i](y)
            y = torch.tanh(y) #F.relu(y)
        
        y = self.hidden_linear[-1](y)
        y = torch.tanh(y)
            
        return y

In [95]:
mlp = MLP(40, [50, 50, 50], 1)

In [96]:
#mlp(torch.rand(1,40))

In [97]:
# # Hyperparameters
batch_size = 50
lr = 0.001
weight_decay = 5e-4
num_epochs = 800

In [98]:
data_loader_X_train = torch.utils.data.DataLoader(
    X_train,
    shuffle=False,
    batch_size = batch_size
)

data_loader_y_train = torch.utils.data.DataLoader(
    y_train,
    shuffle = False,
    batch_size = batch_size
)

In [99]:
optimizer = torch.optim.Adam(mlp.parameters(), lr=lr, weight_decay=weight_decay, amsgrad = True)

In [103]:
mlp.train()
f_loss = nn.MSELoss() #(preds-y_train[i])**2

for epoch in range(num_epochs):
    total_loss= 0
#     total_correct =0
    
    iter_y_train = iter(data_loader_y_train)
    iter_X_train = iter(data_loader_X_train)
    
    #Data
    for batch in iter_X_train:
        #Prediction
        preds = mlp(batch)

        #Calculate the loss
        loss = f_loss(preds.reshape(batch_size,1), next(iter_y_train).reshape(batch_size,1))
        
        total_loss += loss
    
    #Backpropagate
    optimizer.zero_grad() #To avoid adding up gradients
    total_loss.backward() #calculate gradients

    #Optimizer step
    optimizer.step() #Update weights
        
    if epoch%50 == 0:
        print("epoch ", epoch, " total loss  ", total_loss )
    
    if epoch == num_epochs-1:
        print("epoch ", epoch, " total loss  ", total_loss )

epoch  0  total loss   tensor(0.4967, grad_fn=<AddBackward0>)
epoch  50  total loss   tensor(0.4836, grad_fn=<AddBackward0>)
epoch  100  total loss   tensor(0.4703, grad_fn=<AddBackward0>)
epoch  150  total loss   tensor(0.4641, grad_fn=<AddBackward0>)
epoch  200  total loss   tensor(0.4485, grad_fn=<AddBackward0>)
epoch  250  total loss   tensor(0.4871, grad_fn=<AddBackward0>)
epoch  300  total loss   tensor(0.4602, grad_fn=<AddBackward0>)
epoch  350  total loss   tensor(0.4325, grad_fn=<AddBackward0>)
epoch  400  total loss   tensor(0.4589, grad_fn=<AddBackward0>)
epoch  450  total loss   tensor(0.4454, grad_fn=<AddBackward0>)
epoch  500  total loss   tensor(0.4417, grad_fn=<AddBackward0>)
epoch  550  total loss   tensor(0.4330, grad_fn=<AddBackward0>)
epoch  600  total loss   tensor(0.4351, grad_fn=<AddBackward0>)
epoch  650  total loss   tensor(0.4341, grad_fn=<AddBackward0>)
epoch  700  total loss   tensor(0.4340, grad_fn=<AddBackward0>)
epoch  750  total loss   tensor(0.4166, gra

In [104]:
mlp(X_test[1:10])

tensor([[ 0.4542],
        [ 0.2443],
        [ 0.3621],
        [ 0.4023],
        [ 0.2997],
        [ 0.3387],
        [ 0.4273],
        [ 0.3234],
        [-0.0033]], grad_fn=<TanhBackward>)

In [89]:
y_test[1:10]

tensor([ 0.4086,  0.0567,  0.3450,  0.4014,  0.3403,  0.3413,  0.4215,  0.4311,
        -0.1600])

In [107]:
#R squared 
r2_score( y_train.detach().numpy() , mlp(X_train).detach().numpy() )

0.6751711843709476

In [108]:
#R squared
r2_score( y_test.detach().numpy() , mlp(X_test).detach().numpy() )

0.6663847495652011